In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from scipy.spatial import distance
from sklearn import preprocessing
import umap

from rdkit import Chem
from rdkit.Chem import AllChem

import plotly.graph_objects as go

# 1 Generating all designed reaction conditions

## 1.1 Build DataFrame of selected CNPs and ligands

In [ ]:
# Building dataframe for Ni ligands and calculating fingerprints using the molecular structure files
ligands_dir = Path('../data/Ni-27-ligands')
fps_kwargs = {
    'radius': 3,
    'fpSize':2048
}
fpgen = AllChem.GetMorganGenerator(**fps_kwargs)
i = 0
df_ligands = pd.DataFrame(columns=['name', 'SMILES', 'fps'])
for file in ligands_dir.iterdir():
    if file.suffix == '.pdb':
        mol = Chem.MolFromPDBFile(file.as_posix())
        df_ligands.loc[i, :'SMILES'] = [int(file.stem), Chem.MolToSmiles(mol)]
        df_ligands.at[i, 'fps'] = fpgen.GetFingerprint(mol).ToList()
        i += 1
df_ligands = df_ligands.sort_values(by='name', ignore_index=True)

In [ ]:
# Built dataframe for ligands and calculate fingerprints
cnps_idx = [122, 127, 128, 129, 131, 187, 234, 239, 240, 243, 295, 323, 379, 439, 463, 464, 491, 519]
dft_df = pd.read_excel('../data/560_DFT_result.xlsx')
df_cnps = dft_df.loc[cnps_idx, ['name', 'SMILES', 'ID', 'EA']]
df_cnps.loc[:, 'fps'] = pd.DataFrame(columns=['fps']).astype('object')
for idx in df_cnps.ID:
    mol = Chem.MolFromSmiles(df_cnps.loc[idx, 'SMILES'])
    df_cnps.at[idx, 'fps'] = fpgen.GetFingerprint(mol).ToList()
df_cnps = df_cnps.sort_index().reset_index(drop=True)

## 1.2 Build the DataFrame of designed conditions

In [ ]:
# Generating all designed reaction conditions as a dataframe
reaction_conditions = []
df_conditions = pd.DataFrame(columns=['idx', 'name', 'ligand', 'molecule_id', 'Ni'])
i =  0
for ligand_idx in range(len(df_ligands)):
    for mol_idx in range(len(df_cnps)):
        for ni_idx in range(len(amount_ni)):
            cond = [ligand_idx, mol_idx, ni_idx]
            reaction_conditions.append(cond)
            name = 'L{}_{}_{}'.format(df_ligands.loc[ligand_idx, 'name'], df_cnps.loc[mol_idx, 'name'], (ni_idx + 1))
            df_conditions.loc[i, :] = [cond, name, df_ligands.loc[ligand_idx, 'name'], str(df_cnps.loc[mol_idx, 'ID']), (ni_idx + 1)]
            i += 1

In [ ]:
# Saving data
df_ligands.to_pickle('../data/opt_conditions/df_ligands.pkl')
df_cnps.to_pickle('../data/opt_conditions/df_selected_molecules.pkl')
df_conditions.to_pickle('../data/opt_conditions/df_reaction_conditions.pkl')

# 2 Build the distance matrix of designed conditions

In [ ]:
# Loading data and define fps distance
df_ligands = pd.read_pickle('../data/opt_conditions/df_ligands.pkl')
df_cnps = pd.read_pickle('../data/opt_conditions/df_selected_molecules.pkl')
df_conditions = pd.read_pickle('../data/opt_conditions/df_reaction_conditions.pkl')

def cal_jaccard_dis_matrix(df, feature='fps'):
    dis_matrix = np.zeros((df.shape[0], df.shape[0]))
    for i in df.index:
        for j in df.index:
            dis = distance.jaccard(df.loc[i, feature], df.loc[j, feature])
            dis_matrix[i][j] = dis
    return dis_matrix

In [ ]:
# Load scaled CNPs EA distance matrix
cnps_ea_matrix = np.load('../data/opt_conditions/cnps_ea_matrix.npy')
# Ni amount and distance
amount_ni = preprocessing.minmax_scale([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]).reshape(-1, 1)
ni_matrix = distance.cdist(amount_ni, amount_ni, metric='euclidean')
# Calculating jaccard fingerprints distance of ligands and molecules
ligand_matrix = cal_jaccard_dis_matrix(df=df_ligands)
mol_matrix = cal_jaccard_dis_matrix(df=df_cnps)

In [ ]:
# Using a list as the iterable obj to speed up the build process
reaction_conditions = df_conditions.idx.to_list()
# Build the distance matrix of reaction conditions, shape = (4, 4500, 4500)
dis_matrix = np.zeros((4, len(reaction_conditions), len(reaction_conditions)))
for idx_0, condition_0 in enumerate(reaction_conditions):
    for idx_1, condition_1 in enumerate(reaction_conditions):
        dis_matrix[0,idx_0, idx_1] = ligand_matrix[condition_0[0]][condition_1[0]]  # Fingerprints distance of ligands
        dis_matrix[1,idx_0, idx_1] = mol_matrix[condition_0[1]][condition_1[1]]     # Fingerprints distance of CNPs
        dis_matrix[2,idx_0, idx_1] = cnps_ea_matrix[condition_0[1]][condition_1[1]] # EA distance of CNPs
        dis_matrix[3,idx_0, idx_1] = ni_matrix[condition_0[2]][condition_1[2]]      # The distance of Ni loading 

In [ ]:
np.save('../data/opt_conditions/dis_matrix.npy', dis_matrix)

# 3 Visualisation using UMAP and Plotly

In [ ]:
# Loading required data
dis_matrix = np.load('../data/opt_conditions/dis_matrix.npy')
random_selection = pd.read_excel('../data/opt_conditions/exp_results.xlsx', sheet_name='random', index_col=0)
random_selection.head()

In [ ]:
# UMAP parameters
umap_kwargs = {
    'metric': 'precomputed', 
    'min_dist': 1, 
    'n_neighbors': 600, 
    'n_components': 2, 
    'n_epochs': 500,
    'random_state': 1
}
# Set the kernel ratio parameters
a = np.array([0.63537337, 1.10563238, 0.50150809, 0.94269454]).reshape(4, -1, 1)
condition_dis_matrix = sum(dis_matrix*a)
# Calculation of UMAP coordinates
up = umap.UMAP(**umap_kwargs)
pos_umap = up.fit_transform(condition_dis_matrix)
# Saving the coordinates to dataframe
df_conditions.loc[:, 'pos_0'] = pos_umap[:, 0]
df_conditions.loc[:, 'pos_1'] = pos_umap[:, 1]
# define the selected and unselected DataFrame
selected_idx = random_selection.index.values
df_conditions.Ni = df_conditions.Ni.astype(dtype='int')
df_selection = df_conditions.loc[selected_idx, :]
df_unselected = df_conditions.drop(index=selected_idx)

In [ ]:
# Making figures
axis_template = dict(showgrid=False, zeroline=False, showline=False, showticklabels=False)
fig_val = go.Figure()
fig_val.add_trace(go.Scatter(
    x=df_unselected.loc[:, 'pos_0'],
    y=df_unselected.loc[:, 'pos_1'],
    mode='markers',
    name='Designed chemical space',
    text=df_unselected.loc[:, 'molecule_id'],
    marker=dict(
        symbol='circle',
        size=5,
        opacity=0.7,
        line=dict(color='white', width=1), 
        color='Gray'
    )
))
fig_val.add_trace(go.Scatter(
    x=df_conditions.loc[selected_idx, 'pos_0'],
    y=df_conditions.loc[selected_idx, 'pos_1'],
    mode='markers',
    name='Random selection',
    text=df_conditions.loc[selected_idx, 'molecule_id'],
    marker=dict(
        symbol='diamond',
        size=random_selection.loc[:, 'yield']*20+10,
        opacity=0.7,
        line=dict(color='white', width=1), 
        color='Black'
    )
))
fig_val.update_layout(
        margin = dict(l=0, r=230, t=0, b=0),
        font = dict(family='Arial',
            size=14,
            color='black'),
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=axis_template,
        yaxis=axis_template,
        showlegend=True,
        width=800,
        height=600
    )